In [1]:
%load_ext watermark
%watermark -a 'Sebastian Raschka' -v -p torch

Sebastian Raschka 

CPython 3.6.7
IPython 6.5.0

torch 0.4.1


- Runs on CPU or GPU (if available)

# Rank Softmax Regression

Implementation of rank softmax regression (multinomial logistic regression).

In [2]:
def rank_cross_entropy(logits, targets, num_classes, reduction='elementwise_mean'):
    smax = F.softmax(logits, dim=1)
    
    weights = torch.abs(targets - torch.argmax(logits, dim=1) + 1).float() / logits.shape[1]
    expanded_targets = (torch.arange(num_classes) == targets[:, None]).float()
    val = - torch.sum(torch.log(smax) * expanded_targets * weights[:, np.newaxis], dim=1)
    if reduction == 'elementwise_mean':
        return torch.mean(val)
    elif reduction == 'elementwise_sum':
        return torch.sum(val)
    elif reduction == 'none':
        return val
    else:
        raise NotImplemented

## Imports

In [3]:
import numpy as np
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch

## Settings and Dataset

In [4]:
##########################
### SETTINGS
##########################

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Hyperparameters
random_seed = 123
learning_rate = 0.2
num_epochs = 10
batch_size = 256

# Architecture
num_features = 784
num_classes = 10


##########################
### MNIST DATASET
##########################

train_dataset = datasets.MNIST(root='data', 
                               train=True, 
                               transform=transforms.ToTensor(),  
                               download=True)

test_dataset = datasets.MNIST(root='data', 
                              train=False, 
                              transform=transforms.ToTensor())


train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=batch_size, 
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size, 
                         shuffle=False)


# Checking the dataset
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

Image batch dimensions: torch.Size([256, 1, 28, 28])
Image label dimensions: torch.Size([256])


In [5]:
##########################
### MODEL
##########################

class SoftmaxRegression(torch.nn.Module):

    def __init__(self, num_features, num_classes):
        super(SoftmaxRegression, self).__init__()
        self.linear = torch.nn.Linear(num_features, num_classes)
        
        self.linear.weight.detach().zero_()
        self.linear.bias.detach().zero_()
        
    def forward(self, x):
        logits = self.linear(x)
        probas = F.softmax(logits, dim=1)
        return logits, probas

model = SoftmaxRegression(num_features=num_features,
                          num_classes=num_classes)

model.to(device)

##########################
### COST AND OPTIMIZER
##########################


optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [6]:
# Manual seed for deterministic data loader
torch.manual_seed(random_seed)


def compute_accuracy(model, data_loader):
    correct_pred, num_examples = 0, 0
    
    for features, targets in data_loader:
        features = features.view(-1, 28*28).to(device)
        targets = targets.to(device)
        logits, probas = model(features)
        _, predicted_labels = torch.max(probas, 1)
        num_examples += targets.size(0)
        correct_pred += (predicted_labels == targets).sum()
        
    return correct_pred.float() / num_examples * 100
    

for epoch in range(num_epochs):
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.view(-1, 28*28).to(device)
        targets = targets.to(device)
            
        ### FORWARD AND BACK PROP
        logits, probas = model(features)
        cost = rank_cross_entropy(logits, targets, num_classes=10)
        optimizer.zero_grad()
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f' 
                   %(epoch+1, num_epochs, batch_idx, 
                     len(train_dataset)//batch_size, cost))
            
    print('Epoch: %03d/%03d training accuracy: %.2f%%' % (
          epoch+1, num_epochs, 
          compute_accuracy(model, train_loader)))

Epoch: 001/010 | Batch 000/234 | Cost: 1.2997
Epoch: 001/010 | Batch 050/234 | Cost: 0.2255
Epoch: 001/010 | Batch 100/234 | Cost: 0.1610
Epoch: 001/010 | Batch 150/234 | Cost: 0.1292
Epoch: 001/010 | Batch 200/234 | Cost: 0.1356
Epoch: 001/010 training accuracy: 87.31%
Epoch: 002/010 | Batch 000/234 | Cost: 0.1270
Epoch: 002/010 | Batch 050/234 | Cost: 0.1446
Epoch: 002/010 | Batch 100/234 | Cost: 0.1317
Epoch: 002/010 | Batch 150/234 | Cost: 0.1216
Epoch: 002/010 | Batch 200/234 | Cost: 0.1086
Epoch: 002/010 training accuracy: 88.35%
Epoch: 003/010 | Batch 000/234 | Cost: 0.1074
Epoch: 003/010 | Batch 050/234 | Cost: 0.0920
Epoch: 003/010 | Batch 100/234 | Cost: 0.0876
Epoch: 003/010 | Batch 150/234 | Cost: 0.0884
Epoch: 003/010 | Batch 200/234 | Cost: 0.1159
Epoch: 003/010 training accuracy: 88.92%
Epoch: 004/010 | Batch 000/234 | Cost: 0.1102
Epoch: 004/010 | Batch 050/234 | Cost: 0.0995
Epoch: 004/010 | Batch 100/234 | Cost: 0.0902
Epoch: 004/010 | Batch 150/234 | Cost: 0.1259
Epo

In [7]:
print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader)))

Test accuracy: 90.98%
